# Testing the performance of TMSN

In this notebook, we evaluate TMSN in terms of package loss and round trip time of the packets.

We tested by different sizes of packets, ranging from 1 KB to 200 KB, over the clusters with 3 machines and 10 machines. We tested two different waiting intervals between sending out the packets, 50 ms and 10 ms.

We have following conclusions:

* We didn't observe the packages being dropped in these experiments
* When the interval between sending two packets is larger than the round trip time, the round trip time is in the order of millisecond (1-10 ms)
* When the interval between sending two packets is smaller than the round trip time, the round trip time increase probably due to the crowded network. For the 200 KB packets, the round trip time increased from 10ms to 200ms after the interval is shrinked from 50ms to 10ms.

In [177]:
from io import StringIO
import pandas as pd


def get_stats(log_path, num_machines):
    logs = []
    run_id = 0
    test_id = 0
    with open(log_path) as f:
        for line in f:
            if line.startswith("load_size"):
                run_id += 1
            if line.startswith("start new test"):
                test_id = int(line.split(", ")[1])
            if line.startswith("stress perf"):
                logs.append(str(run_id) + "," + str(test_id) + "," + line)
    df = pd.read_csv(StringIO(''.join(logs)), sep=",",
                     names=["run_id", "test_id", "log", "load", "addr", "total", "msg", "msg_echo", "hb", "hb_echo",
                            "msg_time", "hb_time", "avg_msg", "avg_hb"])

    num_load_sets = len(set(df.load))
    if df.iloc[-1].load == df.iloc[0].load:
        num_load_sets += 2
    msg = df.groupby(["run_id", "load", "test_id"]).sum()[["msg_time", "msg_echo"]]

    df = df[df["addr"] == "local"].drop(columns=["log", "addr"]).set_index(["run_id", "load", "test_id"])

    df["msg_time"] = msg["msg_time"]
    df["avg_msg"] = msg["msg_time"] / msg["msg_echo"]

    df = df.reset_index()
    df_mean = df.groupby(df.index % num_load_sets).mean()
    df_std = df.groupby(df.index % num_load_sets).std()

    df_mean["std_msg"] = df_std["avg_msg"]
    df_mean["std_hb"] = df_std["avg_hb"]
    df_mean = df_mean.drop(columns=["test_id", "msg_time", "hb_time", "run_id"])

    return df_mean

## Description of the columns

* load: load size, unit KB
* total: total number of packets received, including message, echo to message, heartbeat, and echo to heartbeats
* msg, msg_echo, hb, hb_echo: total number of message/echo-to-msg/hb/echo-to-hb packets received
* avg_msg: average round trip time of the message packets
* avg_hb: standard deviation of the round trip time of the message packets
* std_msg: average round trip time of the message packets
* std_hb: standard deviation of the round trip time of the heartbeat packets

In [178]:
get_stats("logs-3machines/logs-0.txt", 3)

,load,total,msg,msg_echo,hb,hb_echo,avg_msg,avg_hb,std_msg,std_hb
0,1024.0,226.0,200.0,0.0,18.0,8.0,662.229500,3856.805556,31.505743,2961.460809
1,5120.0,226.0,200.0,0.0,18.0,8.0,739.748500,327.044444,37.421536,77.011890
2,10240.0,226.0,200.0,0.0,18.0,8.0,1240.232500,296.022222,51.460844,29.332202
3,102400.0,226.0,200.0,0.0,18.0,8.0,1752.856500,317.266667,81.972632,43.644748
4,204800.0,225.1,200.0,0.0,17.4,7.7,3242.232874,365.094444,278.616810,75.221839


In [179]:
get_stats("logs-10machines/logs-0.txt", 10)

,load,total,msg,msg_echo,hb,hb_echo,avg_msg,avg_hb,std_msg,std_hb
0,1024.0,988.1,900.0,0.0,80.1,8.0,658.622778,557.981790,51.991438,404.852867
1,5120.0,989.0,900.0,0.0,81.0,8.0,734.341556,67.679012,38.999447,17.739452
2,10240.0,989.0,900.0,0.0,81.0,8.0,1284.516111,62.528395,34.871409,7.508002
3,102400.0,989.0,900.0,0.0,81.0,8.0,1819.284556,72.443210,82.617448,18.406692
4,204800.0,985.5,899.6,0.0,78.2,7.7,4333.769686,360.448824,850.714473,754.148226


In [180]:
get_stats("logs-10machines-2/logs-0.txt", 10)

,load,total,msg,msg_echo,hb,hb_echo,avg_msg,avg_hb,std_msg,std_hb
0,1024.0,989.000000,900.0,0.0,81.000000,8.0,651.975030,871.246914,12.548383,385.993823
1,1024.0,989.000000,900.0,0.0,81.000000,8.0,651.975030,1102.726852,12.548383,502.407457
2,5120.0,989.000000,900.0,0.0,81.000000,8.0,802.033651,58.449735,215.130583,4.649559
3,10240.0,988.857143,900.0,0.0,80.857143,8.0,1247.038254,66.510780,42.473750,10.225594
4,102400.0,988.857143,900.0,0.0,80.857143,8.0,2105.315397,95.655269,314.709083,54.346626
5,204800.0,989.000000,900.0,0.0,81.000000,8.0,11490.021270,718.820106,18690.682046,1647.904972
6,1024.0,989.000000,900.0,0.0,81.000000,8.0,654.096648,578.596120,11.903519,509.238463


In [181]:
get_stats("logs-10ms/logs-0.txt", 10)

,load,total,msg,msg_echo,hb,hb_echo,avg_msg,avg_hb,std_msg,std_hb
0,1024.0,988.625000,900.000000,0.0,80.625000,8.0,2850.096528,106.936380,29.205995,76.582900
1,1024.0,988.875000,900.000000,0.0,80.875000,8.0,2825.145754,137.242612,32.943708,80.056380
2,5120.0,989.000000,900.000000,0.0,81.000000,8.0,693.735079,57.749559,18.962473,8.372982
3,10240.0,989.000000,900.000000,0.0,81.000000,8.0,1201.930952,60.844797,34.057054,7.004958
4,102400.0,989.000000,900.000000,0.0,81.000000,8.0,2799.381905,61.225750,501.641123,8.999796
5,204800.0,989.000000,900.000000,0.0,81.000000,8.0,187150.171270,2303.038801,17686.344034,1516.454149
6,1024.0,957.857143,871.428571,0.0,78.428571,8.0,2921.393401,141.242882,54.995889,96.363354


In [182]:
get_stats("logs-10ms-2/logs-0.txt", 10)

,load,total,msg,msg_echo,hb,hb_echo,avg_msg,avg_hb,std_msg,std_hb
0,1024.0,989.000000,900.000000,0.0,81.000000,8.0,2929.370139,108.324074,54.437642,78.568601
1,1024.0,989.000000,900.000000,0.0,81.000000,8.0,2852.424472,71.385802,63.444558,27.780512
2,5120.0,989.000000,900.000000,0.0,81.000000,8.0,680.324921,60.830688,22.431047,9.038121
3,10240.0,989.000000,900.000000,0.0,81.000000,8.0,1262.979048,65.800705,68.314252,10.230356
4,102400.0,989.000000,900.000000,0.0,81.000000,8.0,3005.560159,88.476190,598.030372,44.326720
5,204800.0,989.000000,900.000000,0.0,81.000000,8.0,245795.790000,4270.770723,12274.920665,2285.502296
6,1024.0,963.285714,876.857143,0.0,78.428571,8.0,2822.740499,91.193752,19.579589,32.645082
